# <center> Evaluation du modèle </center>

In [ ]:
from bigml.api import BigML
from pandas import read_csv
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = read_csv('../handson-ml2/my_predictionsValEnsemble.csv')
df = df.rename(columns={"SeriousDlqin2yrs.1":"MyPrediction"})
df = df.drop(columns={"field1"})
df

In [ ]:
data = read_csv('../handson-ml2/my_predictionsValDeepnet.csv')
data = data.rename(columns={"SeriousDlqin2yrs.1":"MyPrediction"})
data = data.drop(columns={"field1"})
data

## Première étape: identifier les erreurs pour évaluer notre modèle

In [ ]:
TN = 0
TP = 0
FN = 0
FP = 0

serie = df.transpose() # here we convert the DataFrame into a Serie
for ligne in serie:
    if(serie[ligne]['SeriousDlqin2yrs'] == 0 and serie[ligne]['MyPrediction'] == 0):
             TN = TN + 1
            
    if(serie[ligne]['SeriousDlqin2yrs'] == 1 and serie[ligne]['MyPrediction'] == 1):
             TP = TP + 1

    if(serie[ligne]['SeriousDlqin2yrs'] == 1 and serie[ligne]['MyPrediction'] == 0):
             FN = FN + 1

    if(serie[ligne]['SeriousDlqin2yrs'] == 0 and serie[ligne]['MyPrediction'] == 1):
             FP = FP + 1
            
print(f"TN : {TN}")
print(f"TP : {TP}")
print(f"FN : {FN}")
print(f"FP : {FP}")

*<center> Ajouter une colonne erreur contenant les TN,FN, TP, FP</center>*

In [ ]:
def confusion(row):
    if row['SeriousDlqin2yrs'] == 0 and row['MyPrediction'] == 0:
        error_value = 'TN'
    if row['SeriousDlqin2yrs'] == 1 and row['MyPrediction'] == 1:
        error_value = 'TP'
    if row['SeriousDlqin2yrs'] == 1 and row['MyPrediction'] == 0:
        error_value = 'FN'
    if row['SeriousDlqin2yrs'] == 0 and row['MyPrediction'] == 1:
        error_value = 'FP'
    return error_value

df['error'] = df.apply(confusion, axis = 1)
df.to_csv('../handson-ml2/kaggle-give-me-credit-Ensemble_confusion.csv', index=False)

In [ ]:
df = read_csv('../handson-ml2/kaggle-give-me-credit-Ensemble_confusion.csv')
df

In [ ]:
def confusion(row):
    if row['SeriousDlqin2yrs'] == 0 and row['MyPrediction'] == 0:
        error_value = 'TN'
    if row['SeriousDlqin2yrs'] == 1 and row['MyPrediction'] == 1:
        error_value = 'TP'
    if row['SeriousDlqin2yrs'] == 1 and row['MyPrediction'] == 0:
        error_value = 'FN'
    if row['SeriousDlqin2yrs'] == 0 and row['MyPrediction'] == 1:
        error_value = 'FP'
    return error_value

data['error'] = data.apply(confusion, axis = 1)
data.to_csv('../handson-ml2/kaggle-give-me-credit-Deepenet_confusion.csv', index=False)

In [ ]:
data = read_csv('../handson-ml2/kaggle-give-me-credit-Deepenet_confusion.csv')
data

*<center> Mesurer l'accuracy, compter les différentes erreurs, et construire une matrice de confusion</center>*

In [ ]:
accuracy = (TN+TP)/(TP+TN+FP+FN)
accuracy

In [ ]:
matrice = df['error'].value_counts()
matrice

In [ ]:
df = pd.DataFrame(df, columns=['SeriousDlqin2yrs','MyPrediction'])

confusion_matrix = pd.crosstab(df['SeriousDlqin2yrs'], df['MyPrediction'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

*<center> Calculer le gain à partir d'un seuil </center>*

In [ ]:
def gain_seuil(seuil):
    def seuil_pred(row, seuil):
        if row['1 probability'] > seuil and row['SeriousDlqin2yrs'] == 0:
            row['error'] = 'FP'
        if row['1 probability'] > seuil and row['SeriousDlqin2yrs'] == 1:
            row['error'] = 'TP'
        if row['1 probability'] < seuil and row['SeriousDlqin2yrs'] == 0:
            row['error'] = 'TN'
        if row['1 probability'] < seuil and row['SeriousDlqin2yrs'] == 1:
            row['error'] = 'FN'
        return row

    for dataframe in [df]:
        dataframe['error'] = df[['error', '1 probability', 'SeriousDlqin2yrs']].apply(seuil_pred, axis=1, seuil = seuil)

    count = dataframe['error'].value_counts()
    gain = 1
    if "TP" not in  dataframe['error']:
        if "FP" not in  dataframe['error']:
            gain = count[0]*500 - count[1]*2500
        else:
            gain = count[0]*500 - count[1]*2500 - count[3]*500
    else:
        gain = count[0]*500 - count[1]*2500 - count[2]*500

    return gain

In [ ]:
%matplotlib inline

In [ ]:
table = [gain_seuil(n/100) for n in range(1,100)]
seuil = [(n/100) for n in range(1,100)]
plt.plot(table)
plt.ylabel("Seuil")
plt.xlabel("Gain")
plt.title('Gain maximisé')
max_val = max(table)
max_threshold = table.index(max_val)
print(f'Le gain maximum est {max_val} et il est obtenu avec un seuil de {max_threshold}')

*<center> Récupérer les 100 plus grosses erreurs </center>*

In [ ]:
df['absolute_error'] = (df['1 probability']-df['SeriousDlqin2yrs']).abs()
desc_order = df.sort_values(by='absolute_error', ascending = False).head(100)
desc_order.to_csv('../handson-ml2/100error.csv', index=False)

*<center> Calculer l'AUC </center>*

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
score = roc_auc_score(df['SeriousDlqin2yrs'].values,df['1 probability'].values)
print(score)

In [ ]:
positif = df.loc[df['SeriousDlqin2yrs'] == 1] # Tout les Positif 
negatif = df.loc[df['SeriousDlqin2yrs'] == 0] # Tout les negatifs 

x = 0
y = 0
# pour chaque 1 proba ( P ) dans toutes les 1 proba (P)
for threshold_pos in positif['1 probability']:
    # pour chaque 1 Proba ( N ) dans toutes les 1 proba ( N )
    for threshold_neg in negatif['1 probability']:
        if threshold_pos > threshold_neg:
            x += 1 
        y += 1 
print(f"L'AUC est égale à {round((x/y)*100,2)}%")

In [ ]:
nombre_supérieur = 0
nombre_iteration = 0
serie = df.transpose() # here we convert the DataFrame into a Serie
for ligne in serie:
    if serie[ligne]['SeriousDlqin2yrs'] == 1: 
        proba_positif = serie[ligne]['1 probability'] # On stocke la valeur de 1 probability à la variable proba_positif
        for ligne_neg in serie: 
            if serie[ligne_neg]['SeriousDlqin2yrs'] == 0:
                if proba_positif > serie[ligne_neg]['1 probability']: # Si la valeur de proba_positif est inférieure à la valeur de 1 probability
                    nombre_supérieur += 1 # On incrémente la valeur de 1
                nombre_iteration += 1 # On incrémente le nombre de boucle total (dénominateur pour le calcul de l'AUC)

print(nombre_supérieur)
print(nombre_iteration)
print(f"{nombre_supérieur/nombre_iteration}")

*<center> Tracer la courbe ROC </center>*

In [ ]:
df = read_csv('../handson-ml2/kaggle-give-me-credit-train_confusion.csv')
#False positive rate
fpr = []
# True positive rate
tpr = []

thresh = [(n/100) for n in range(0,100)]

count = df['error'].value_counts()
number_of_positive = count[2] + count[1]
number_of_negative = count[3] + count[0]

for t in thresh:
    FP = 0
    TP = 0
    for i in range(len(df['1 probability'])):
        if (df['1 probability'][i]>t):
            if df['SeriousDlqin2yrs'][i] == 1:
                TP = TP + 1
            if df['SeriousDlqin2yrs'][i] == 0:
                FP = FP + 1
    fpr.append(FP/float(number_of_negative))
    tpr.append(TP/float(number_of_positive))

plt.plot(fpr,tpr)
plt.ylabel("True positive rate")
plt.xlabel("False positive rate")
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.show()